In [1]:
from utils import *

In [4]:
import os

stl_files = [f for f in os.listdir("/data/SML/midmesh/SE_stl/AD") if f.endswith(".stl")]
print(stl_files)

['AD_UPR_RAIL.stl', 'AD_MOLDING.stl', 'AD_MAIN_TRIM.stl', 'AD_SW_BEZEL.stl', 'AD_GARNISH.stl', 'AD_ARM_REST_PAD.stl', 'AD_UPR_TRIM.stl', 'AD_CTR_TRIM.stl', 'AD_IMPACT_PAD.stl', 'AD_ARM_REST_COVER.stl', 'AD_M_POCKET.stl', 'AD_LWR_TRIM.stl', 'AD_ARM_REST_CORE.stl', 'AD_DT_ALL.stl']


In [ ]:
for i in range(len(stl_files)):
    if i == 2:
        continue
    print(f"Processing file {i+1} / {len(stl_files)}: {stl_files[i]}")
    node, elem = read_mesh(os.path.join("/data/SML/midmesh/SE_stl/AD", stl_files[i]))
    triangles = elem['S3']  
    max_node, max_triangle, max_neighbors = main_pilot(node, triangles, face='max', new_mesh_side_length=3.0, cuda_device='cuda:2')
    min_node, min_triangle, min_neighbors = main_pilot(node, triangles, face='min', new_mesh_side_length=3.0, cuda_device='cuda:2')
    mid_node, mid_triangle = make_mid_pilot(max_node, min_node, max_triangle, max_neighbors, iterations=5, lambda_factor=0.5)
    save_mesh_to_vtp(mid_node, mid_triangle, f"/home/ubuntu/SML/MidMesh/test/{stl_files[i]}_midtest.vtp")

Processing file 1 / 14: AD_UPR_RAIL.stl
Memory info: 66.95GB free, using batch_size=400, grid_size=15
Processing 3648 triangles in 10 batches...
Batch 0/10
Memory info: 66.95GB free, using batch_size=400, grid_size=15
Processing 3648 triangles in 10 batches...
Batch 0/10
Small diff count: 551, Large diff count: 832
Processing file 2 / 14: AD_MOLDING.stl
Memory info: 66.95GB free, using batch_size=400, grid_size=15
Processing 7612 triangles in 20 batches...
Batch 0/20
Batch 10/20
Memory info: 66.95GB free, using batch_size=400, grid_size=15
Processing 7612 triangles in 20 batches...
Batch 0/20


In [ ]:
for i in range(len(stl_files)):
    node, elem = read_mesh(os.path.join("/home/ubuntu/SML/MidMesh/test", f"{stl_files[i]}_midtest.vtp"))
    triangles = elem['S3']  
    if i == 0:
        whole_node = node
        whole_triangle = triangles
    else:
        whole_triangle = np.vstack((whole_triangle, triangles + whole_node.shape[0]))
        whole_node = np.vstack((whole_node, node))

print(whole_node.shape, whole_triangle.shape)
save_mesh_to_vtp(whole_node, whole_triangle, f"/home/ubuntu/SML/MidMesh/AD_whole_midtest.vtp")